# Getting Game IDs

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://fbref.com/en/comps/10/2022-2023/schedule/2022-2023-Championship-Scores-and-Fixtures"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <td> elements with data-stat="score"
score_cells = soup.find_all('td', {'data-stat': 'score'})

# Extract game IDs from the <a> tags within the score_cells, the third element is the game ID
game_ids = [cell.find('a')['href'].split('/')[3] for cell in score_cells if cell.find('a')]
# Then strip the white space in the IDs
game_ids = [x.strip(' ') for x in game_ids]

In [2]:
game_ids = game_ids[0:3]

In [3]:
game_ids

['46d9048f', '03cea9be', '78f52c96']

# Scraping Shot Data

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

for unique_id in game_ids:
    # URL of the page to scrape
    url = f'https://fbref.com/en/matches/{unique_id}/'

    # Get the game ID from the URL
    game_id = url.split("/")[-2]

    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the Shots table on the page
    shots_table = soup.find("div", {"id": "switcher_shots"})

    # Create empty lists to store data and create the columns
    data = []
    headers = ['Game_ID', 'Minute', 'Player', 'Team', 'xG', 'PSxG', 'Result', 'Distance', 'Body Part', 'Notes', 'SCA 1 Player', 'Event 1', 'SCA 2 Player', 'Event 2']

    # Extract table rows
    rows = shots_table.find("tbody").find_all("tr")
    for row in rows:
        row_data = [game_id] + [cell.get_text().strip() for cell in row.find_all(["th", "td"])]
        data.append(row_data)

    # Create a DataFrame from the scraped data
    shots_df = pd.DataFrame(data, columns=headers)

    # Find the home and away teams and formations
    team_headers = soup.find_all("th", {"colspan": "2"})
    teams = [header.text.strip().split(" (")[0] for header in team_headers if "(" in header.text.strip()]
    formations = [header.text.strip().split(" (")[1][:-1] for header in team_headers if "(" in header.text.strip()]

    # Extracting the home and away teams and formations
    home_team = teams[0]
    away_team = teams[1]
    home_formation = formations[0]
    away_formation = formations[1]

    # Find the competition -- only using .find because there are multiple in the HTML and we only need one, then take only the text
    competition = soup.find("a", {"href": "/en/comps/10/2022-2023/2022-2023-Championship-Stats"}).text

    # Extract the match date and add to df, this one is a little tricky as it's hidden within the span tag
    span_venuetime = soup.find("span", class_="venuetime")
    match_date = span_venuetime.get("data-venue-date") if span_venuetime else None

    #Get location, 4th element in that list
    location = soup.find("div", {"class": "scorebox_meta"}).find_all("small")[3].text

    # Create a DataFrame from the scraped data
    shots_df = pd.DataFrame(data, columns=headers)

    # Add the team, away team, formation, and opponent formation information to the DataFrame
    shots_df['Home_Team'] = home_team
    shots_df['Away_Team'] = away_team
    shots_df['Home Formation'] = home_formation
    shots_df['Away Formation'] = away_formation
    shots_df['is_home_shot'] = np.where(shots_df['Team'] == home_team, 1, 0)
    shots_df['is_away_shot'] = np.where(shots_df['Team'] ==  away_team, 1, 0)
    shots_df['location'] = location
    shots_df['match_date'] = match_date

In [7]:
shots_df

,Game_ID,Minute,Player,Team,xG,PSxG,Result,Distance,Body Part,Notes,...,SCA 2 Player,Event 2,Home_Team,Away_Team,Home Formation,Away Formation,is_home_shot,is_away_shot,location,match_date
0,78f52c96,4,Matthew Sorinola,Swansea City,0.03,,Blocked,18,Right Foot,,...,Joël Piroe,Pass (Live),Rotherham Utd,Swansea City,3-1-4-2,3-4-3,0,1,"AESSEAL New York Stadium, Rotherham",2022-07-30
1,78f52c96,7,Joël Piroe,Swansea City,0.02,0.03,Saved,25,Left Foot,,...,Harry Darling,Pass (Live),Rotherham Utd,Swansea City,3-1-4-2,3-4-3,0,1,"AESSEAL New York Stadium, Rotherham",2022-07-30
2,78f52c96,13,Joël Piroe,Swansea City,0.07,,Blocked,21,Left Foot,,...,Michael Obafemi,Pass (Live),Rotherham Utd,Swansea City,3-1-4-2,3-4-3,0,1,"AESSEAL New York Stadium, Rotherham",2022-07-30
3,78f52c96,16,Chiedozie Ogbene,Rotherham Utd,0.06,0.23,Goal,4,Head,,...,,,Rotherham Utd,Swansea City,3-1-4-2,3-4-3,1,0,"AESSEAL New York Stadium, Rotherham",2022-07-30
4,78f52c96,19,Conor Washington,Rotherham Utd,0.06,0.03,Saved,20,Right Foot,Deflected,...,Grant Hall,Pass (Live),Rotherham Utd,Swansea City,3-1-4-2,3-4-3,1,0,"AESSEAL New York Stadium, Rotherham",2022-07-30
5,78f52c96,25,Chiedozie Ogbene,Rotherham Utd,0.11,0.65,Saved,8,Right Foot,Volley,...,Daniel Barlaser,Pass (Live),Rotherham Utd,Swansea City,3-1-4-2,3-4-3,1,0,"AESSEAL New York Stadium, Rotherham",2022-07-30
6,78f52c96,31,Harry Darling,Swansea City,0.01,,Blocked,36,Right Foot,,...,,,Rotherham Utd,Swansea City,3-1-4-2,3-4-3,0,1,"AESSEAL New York Stadium, Rotherham",2022-07-30
7,78f52c96,38,Harry Darling,Swansea City,0.01,0.40,Goal,33,Right Foot,,...,Joël Latibeaudière,Pass (Live),Rotherham Utd,Swansea City,3-1-4-2,3-4-3,0,1,"AESSEAL New York Stadium, Rotherham",2022-07-30
8,78f52c96,43,Jamie Paterson,Swansea City,0.02,0.02,Saved,31,Right Foot,,...,Joël Latibeaudière,Pass (Live),Rotherham Utd,Swansea City,3-1-4-2,3-4-3,0,1,"AESSEAL New York Stadium, Rotherham",2022-07-30
9,78f52c96,45+2,Nathan Wood-Gordon,Swansea City,0.05,,Off Target,11,Head,,...,Nathan Wood-Gordon,Fouled,Rotherham Utd,Swansea City,3-1-4-2,3-4-3,0,1,"AESSEAL New York Stadium, Rotherham",2022-07-30
